# Restricted Boltzman Machine 

A restricted Boltzmann Machine is an "Energy Based" generative stochastic model. They were initially invented by Paul Smolensky in 1986 and were called "Harmonium". After the evolution of training algorithms in the mid 2000's by Geoffrey Hinton, the boltzman machine became more prominent. They gained big popularity in recent years in the context of the Netflix Prize where RBMs achieved state of the art performance in collaborative filtering and have beaten most of the competition.

RBM's are useful for dimensionality reduction, classification, regression, collaborative filtering, feature learning and topic modeling. 


## Architecture
RBMs are shallow, two-layer neural nets that constitute the building blocks of deep-belief networks. The first layer of the RBM is called the visible, or input, layer, and the second is the hidden layer. The absense of an output layer is apparent. As we move forward we would learn why the output layer won't be needed.
<img src="figure1.png" width="150" height="50" title="Layers in RBM">
Figure1: Layers in RBM
Each circle in the figure above represents a neuron-like unit called a node, and nodes are simply where calculations take place. 
<img src="figure3.png" width="500" height="200" title="Layers in RBM">
Figure2: Structure of RBM
The nodes are connected to each other across layers, but no two nodes of the same layer are linked. That is, there is no intra-layer communication – this is the restriction in a restricted Boltzmann machine. Each node is a locus of computation that processes input, and begins by making stochastic decisions about whether to transmit that input or not. Each visible node takes a low-level feature from an item in the dataset to be learned.

Let's start by importing the packages that will be required in this project.

In [1]:
import numpy as np
from collections import Counter, defaultdict
import pandas as pd
from scipy.sparse import coo_matrix, hstack
from sklearn.feature_extraction.text import CountVectorizer
np_rng = np.random.RandomState(1234) #setting the random state


In [2]:
# graded
# import data

df = pd.read_excel("amazon.xlsx")

In [3]:
#run this and check if you have got the correct output
df.head()

,Text
0,So there is no way for me to plug it in here i...
1,"Good case, Excellent value."
2,Great for the jawbone.
3,Tied to charger for conversations lasting more...
4,The mic is great.


In Topic Modelling, you find the best set of topics that describe the document. There are various ways to perform topic modelling one of which is RBM. You train your RBM on a set of documents. 
The visible layers will be the words in the text, the hidden layers will give the Topics. 
To input words into the visible layer, let's convert the train and test data you split above into a bag of words model.

In [4]:
#graded
# create bag of words model on train and test data
tf = CountVectorizer(input='content', lowercase=True, token_pattern=r"(?u)\b\w\w+\b", analyzer='word',
                        max_df=50, min_df=1, binary=False, dtype=np.int64)#the final shape should be (number of documents, vocabulary)

# fit tf on the dataframe df
tf = tf.fit(df['Text'])

# transform df dataframe
trainX = tf.transform(df['Text'])

In [5]:
#check if you are getting the correct output
print(sum(trainX.toarray()[1]))
trainX.shape

3


(1000, 1825)

You should get:
<br>

3 <br>
(1000, 1825)

Now that you have the bag of words model, let's define the number of visible and hidden units.

In [6]:
#graded
# define visible units
visibleUnits = trainX.shape[1]# vocabulary size ~1 line

# assign number of units
hiddenUnits = 5 # hyperparameter, this means that we are looking for 5 topics

In [7]:
visibleUnits

1825

In [8]:
#graded
# utility Functions

# deine the sigmoid function
def sigmoid(X):
    return 1.0/(1.0 + np.exp(-1.0*X))# ~ 1 line

## RBM as a Probabilistic Model
Restricted Boltzmann Machines are probabilistic. As opposed to assigning discrete values the model assigns probabilities. At each point in time the RBM is in a certain state. The state refers to the values of neurons in the visible and hidden layers v and h. The probability that a certain state of v and h can be observed is given by the following joint distribution:
<img src="figure4.png" width="200" height="70" title="Layers in RBM">

Eq. 2. Joint Distribution for v and h.
Here Z is called the ‘partition function’ that is the summation over all possible pairs of visible and hidden vectors.

The joint distribution is known as the Boltzmann Distribution which gives the probability that a particle can be observed in the state with the energy E. Unfortunately it is very difficult to calculate the joint probability due to the huge number of possible combination of v and h in the partition function Z. Much easier is the calculation of the conditional probabilities of state h given the state v and conditional probabilities of state v given the state h:
<img src="figure5.png" width="200" height="70" title="Layers in RBM">

Eq. 3. Conditional probabilities for h and v.
It should be noticed beforehand (before demonstrating this fact on practical example) that each neuron in a RBM can only exist in a binary state of 0 or 1. The most interesting factor is the probability that a hidden or visible layer neuron is in the state 1 — hence activated. 

# Contrastive Divergence

### Gibbs Sampling
The first part of the training is called Gibbs Sampling. Given an input vector v we are using p(h|v) for prediction of the hidden values h via sampling. Knowing the hidden values we use p(v|h) for prediction of new input values v via sampling. This process is repeated k times. After k iterations, we obtain the visible vector $v_k$ which was recreated from original input values $v_0$.
<img src="figure8.png" width="500" height="300" title="Layers in RBM">

The gibbs function *gibbs* is divided into subparts: <br>
1.*sampleHiddenLayer * <br>
2.*sampleVisibleLayer*

Let's look at *sampleHiddenLayer* now.

### Sample Hidden Layer

You already know that given an input vector v the probability for a single hidden neuron j being activated is:
<img src="figure6.png" width="400" height="200" title="Layers in RBM">

Eq. 4
Here is σ the Sigmoid function.

*sampleHiddenLayer* takes the visible layer as input to calculate the hidden layer using Eq. 4 *h1Pdf* and then samples it to get * h1_sample*

    v_sample: given visible layer matrix; matrix because a batch of data points will be trained at one go
    returns a sample vector of hidden layer and its distribution for a batch of data points
    
    hPdf: distribution of hidden layer; a matrix for batch of datapoints = p(h|v)
    h_sample: sampled hidden layer matrix

In [9]:
#graded
def sampleHiddenLayer(v_sample):
    
    # write the code for calculation of hPdf using vectorized implementation of Eq 4
    hPdf = sigmoid(np.dot(v_sample, W) + hiddenBias)# ~ 1 line
    
    # Here, np.random.binomial is used to create the hidden layer sample matrix
    h_sample = np_rng.binomial(size=hPdf.shape, n=1, p=hPdf)
    return [hPdf, h_sample]

### Sample Visible Layer
Similarly, the probability that a binary state of a visible neuron i is set to 1 is:
<img src="figure7.png" width="400" height="200" title="Layers in RBM">
Eq. 5

As seen in equation 5, we will be writing a function to sample the Visible Layer.
This function samples the visible layer based on the sampled data of hidden layer. <br>

There are some differences in writing the function *sampleVisibleLayer*. <br>Firstly, we use np.random.multinomial to sample the visible layer *v_sample* from the distribution *vPdf*. <br>Secondly,elements of *vPdf* needs to sum to 1 as the function np.random.multinomial used to sample the visible layer takes on probability distributions as *pvals*. In other words, you are finding the softmax values. <br> Thirdly, we also make use of the *D* to sample the visible layer as each document has different word count.
    
    h_sample: given hidden layer matrix; matrix because a batch of data points will be trained at one go
    D: array of the sum of the row of the data vector; vector containing number of words in each document
    
    returns a sample vector of hidden layer and its distribution for a batch of data points
    
    vPdf: distribution of visible layer; a matrix for batch of datapoints = p(v|h)
    v_sample: sampled visible layer matrix
    

In [10]:
#graded
def sampleVisibleLayer(h_sample, D):
    
    # complete the following function such that vPdf has the sum of entries equal to 1 for each of the datapoints in the batch
    # you have to use axis = 1 in writing the denominator
    numerator = np.exp(np.dot(h_sample, W.T) + visibleBias) # ~1 line
    denominator = numerator.sum(axis=1).reshape((batchSize, 1))# ~1 line
    vPdf = numerator/denominator  # ~1 line np.matmul
   
    # Here np.random.multinomial is used to sample as each document has different number of words 
    # and hence D is also a parameter in sampling
    v_sample = np.zeros((batchSize, vPdf.shape[1]))
    
    for i in range(batchSize):
        v_sample[i] = np_rng.multinomial(size=1, n=D[i], pvals=vPdf[i])
        
    return [vPdf, v_sample]

### Sampling

We use the above functions to write the function *gibbs* to run one iteration of gibbs sampling. Note that we are calculating the visible layer samples first and then using it to calculate he hidden layer sample. It'll become clear soon why we are doing so when you write the function for Contrastive Divergence.
    
    Input:
    h_sample: given hidden layer matrix; matrix because a batch of data points will be trained at one go
    D: array of the sum of the row of the data vector; vector containing number of words in each document
    
    Output:
    vPdf: distribution of visible layer
    v_sample: sampled visible layer matrix
    hPdf: distribution of hidden layer
    h_sample: sampled hidden layer matrix

In [11]:
#graded
def gibbs(h_sample, D):    
    #use sampleVIsibleLayer and sampleHiddenLayer 
    vPdf, v_sample = sampleVisibleLayer(h_sample, D)# ~1 line
    hPdf, h_sample = sampleHiddenLayer(v_sample)# ~1 line
    return [vPdf, v_sample, hPdf, h_sample ]


### Contrastive Divergence

You have learned that Contrastive Divergence updates weights after one iteration of Gibbs Sampling. Here, we shall perform *k* such iterations in Contrastive Divergence. 
The update of the weight matrix happens post the Contrastive Divergence step. 


Now, we will writing a funtion to run the contrastive divergence algorithm in k steps
    
    Input:
    data: batch data (visible layer)
    k: no of iterations for gibbs sampling
    
    Output:
    hiddenPDF_data: distribution of the hidden layer based on data
    visibleSamples: visible samples generated by gibbs sampling 
    hiddenPDF: distribution of the hidden layer based on samples generated by gibbs

In [12]:
#graded
def cd_k(data,k):
    D = data.sum(axis=1)
    hiddenPDF_data, hiddenSample_data = sampleHiddenLayer(data)# sample the hidden layer using the input data
    chain_start = hiddenSample_data

    for step in range(k):
        if step == 0:
            visiblePDF, visibleSamples, hiddenPDF, hiddenSamples  = gibbs(chain_start, D)# perform gibbs sampling using chain_start
        else:
            visiblePDF, visibleSamples, hiddenPDF, hiddenSamples = gibbs(hiddenSamples, D)# perform gibbs sampling using hiddenSamples
    return hiddenPDF_data, visibleSamples, hiddenPDF


## Training the Model

Vectors $v_0$ and $v_k$ are used to calculate the activation probabilities for hidden layers $p(h_0|v_0)$ and $p(h_k|v_k)$ (Eq.4). The difference between the outer products of those probabilities with input vectors $v_0$ and $v_k$ results in the update matrix:
<img src="figure9.png" width="300" height="200" title="Layers in RBM">
Eq. 6. Update matrix. **Figure out** the vectorized implementation for this.

In order to calculate $\Delta (bias)$, <br>
<center>$\Delta (visiblebias) = average\_ across\_ batch(v_0 - v_k)$ </center> 
<center>$\Delta (hiddenbias) = average\_across\_ batch(p(h_0|v_0) - p(h_k|v_k))$ </center> 

Using the update matrix the new weights can be calculated with momentum gradient ascent, given by:
<center>  $mW_t = \gamma \ mW_{t-1} - \Delta W$</center> 
<center>  $mvisiblebias_t = \gamma \ mvisiblebias_{t-1} - \Delta visiblebias$</center>
<center>  $mhiddenbias_t = \gamma \ mhiddenbias_{t-1} - \Delta hiddenbias$</center><br>
<center>  $W_t = W_{t-1} + \alpha \ mW_t$</center> 
<center>  $visiblebias_t = visiblebias_{t-1} + \alpha \ mvisiblebias_t$</center> 
<center>  $hiddenbias_t = hiddenbias_{t-1} + \alpha \ mhiddenbias_t$</center> 


Eq. 7. Update rule for the weights.

Note that in the code implementation below <br>
 hiddenPDF_data is $p(h_0|v_0)$ <br>
 visibleSamples is $v_k$ <br>
 hiddenPDF is $p(h_k|v_k)$ <br>
 mdata is $v_0$ <br>
 eta is $\alpha$ <br>
 mrate is $\gamma$ <br>
These will be helpful in writing the weight updates.

In this we will write a function which iterates over our data for *epochs*.
At every epoch we shuffle the data and then run CD on a mini batch size defined by *batchSize*

### Initializations

In [13]:
"""
visibleUnits: no of words in your Bag of words Model
hiddenUnits: no of topics
batchSize: data slice to be selected 
epochs: no of iterations
eta: learning rate
mrate: momentum coefficient
W : weights between the visible and hidden layer
visibleBias, hiddenBias: biases for visible and hidden layer respectively
"""

# define batch size
batchSize = 200

epochs = 100
eta = 0.05 
mrate = 0.5

# initialise weights
weightinit=0.01
W = weightinit * np_rng.randn(visibleUnits, hiddenUnits)
visibleBias = weightinit * np_rng.randn(visibleUnits)
hiddenBias = np.zeros((hiddenUnits))

# we shall use 10 interations of gibbs sampling
k=10

### Training

In [14]:
#graded
def train(dataX,k):
    global W,visibleBias,hiddenBias,mrate,batchSize,epochs # calling the variables initiliazed at the begining
    
    mW = np.asarray( np_rng.uniform(
                    low=-1 * np.sqrt(2. / (hiddenUnits + visibleUnits)),
                    high=1 * np.sqrt(2. / (hiddenUnits + visibleUnits)),
                    size=(visibleUnits, hiddenUnits)
                ),dtype=np.float64)# initialise momentum_weights
    mvisibleBias =  np.zeros(visibleUnits)# initialise momentum_visiblebiases
    mhiddenBias =  np.zeros(hiddenUnits)# initialise momentum_hiddenbiases
    for epoch in range(epochs):
        print("epochs #" , epoch)
        np_rng.shuffle(dataX) #shuffling the data
        
        for i in range(0, dataX.shape[0], batchSize):
            mData = dataX[i:i+batchSize]#select a batch of datapoints
                        
            hiddenPDF_data, visibleSamples, hiddenPDF =  cd_k(mData,k)# perfrom Contrastive Divergence on the batch for k iterations
            
            mW = mrate*mW - (np.dot(mData.T,hiddenPDF_data) - np.dot(mData.T,hiddenPDF))#write the momentum update equation for weight matrix
            
            mvisibleBias =  mrate*mvisibleBias - np.mean(mData-visibleSamples, axis=0)#write the momentum update equation for visiblebias vector
            mhiddenBias =  mrate*mhiddenBias - np.mean(hiddenPDF_data - hiddenPDF, axis=0) #write the momentum update equation for hiddenbias vector

            W = W + eta*mW #weight update equation
            visibleBias = visibleBias + eta*mvisibleBias #visible bias update equation
            hiddenBias = hiddenBias + eta*mhiddenBias #hidden bias update equation)

## Train the Model

This will take around 10 minutes of time to run.

In [15]:
train(trainX.toarray(),k)

epochs # 0
epochs # 1
epochs # 2
epochs # 3
epochs # 4
epochs # 5
epochs # 6
epochs # 7
epochs # 8
epochs # 9
epochs # 10
epochs # 11
epochs # 12
epochs # 13
epochs # 14
epochs # 15
epochs # 16
epochs # 17
epochs # 18
epochs # 19
epochs # 20
epochs # 21
epochs # 22
epochs # 23
epochs # 24
epochs # 25
epochs # 26
epochs # 27
epochs # 28
epochs # 29
epochs # 30
epochs # 31
epochs # 32
epochs # 33
epochs # 34
epochs # 35
epochs # 36
epochs # 37
epochs # 38
epochs # 39
epochs # 40
epochs # 41
epochs # 42
epochs # 43
epochs # 44
epochs # 45
epochs # 46
epochs # 47
epochs # 48
epochs # 49
epochs # 50
epochs # 51
epochs # 52
epochs # 53
epochs # 54
epochs # 55
epochs # 56
epochs # 57
epochs # 58
epochs # 59
epochs # 60
epochs # 61
epochs # 62
epochs # 63
epochs # 64
epochs # 65
epochs # 66
epochs # 67
epochs # 68
epochs # 69
epochs # 70
epochs # 71
epochs # 72
epochs # 73
epochs # 74
epochs # 75
epochs # 76
epochs # 77
epochs # 78
epochs # 79
epochs # 80
epochs # 81
epochs # 82
epochs # 83
ep

# Word Distribution based on Topics
In this function, we are finding the distribution of words over the topics. You can take a look at the words under each topic and see what they are talking about. The number of topics is the number of neurons in the hidden layer. <br>
<br>

For each topic, the function prints the top 15 words that describe the topic. You can see that some of the words occur in multiple topics.

    topic: number of hidden layers
    voc: indexing of the vocabulary
    
Feel free to change the number of iterations of gibbs sampling in Contrastive Divergence and see how the distribution of words change under the topics.

In [16]:
topic_words = defaultdict(list)
def worddist( topic, voc):
    
    """
    Initialize every topic =1 once 
    """
    global topic_words
    vecTopics = np.zeros((topic, topic))
    for i in range(len(vecTopics)):
        vecTopics[i][i] = 1
    
    for i, vecTopic in enumerate(vecTopics):
       
        numerator = np.exp(np.dot(vecTopic, W.T) + visibleBias)
        denominator = numerator.sum().reshape((1, 1))
        word_distribution = (numerator/denominator).flatten()
        
        tmpDict = {}
        for j in voc.keys():
            tmpDict[j] = word_distribution[voc[j]]
        print('topic', str(i), ':', vecTopic)
        lm=0
        for word, prob in sorted(tmpDict.items(), key=lambda x:x[1], reverse=True):
            print ( word, str(prob))
            topic_words[i].append(word)
            lm+=1
            if lm==15:
                break
        print('\n')

In [17]:
worddist( hiddenUnits, tf.vocabulary_)

topic 0 : [1. 0. 0. 0. 0.]
procedure 0.000634285977175353
specially 0.0006340428409141284
casing 0.0006336596674772878
course 0.0006331009132529704
350 0.00063255953436549
build 0.0006319170288527369
messaging 0.0006313938582913948
disapointing 0.0006307029366793908
aspect 0.0006296386442826928
christmas 0.0006294881207015874
bluetooths 0.0006293073547480014
top 0.0006291686031157599
funny 0.0006286690106485316
thanks 0.0006286143885703104
row 0.000627704406980423


topic 1 : [0. 1. 0. 0. 0.]
procedure 0.000642465568151984
windows 0.000639532678133595
drains 0.0006362136895341591
funny 0.0006332184449383049
disapointing 0.0006326540249438052
moving 0.0006322968963544224
source 0.0006319827556500726
shifting 0.0006310035212463083
fully 0.0006308711074326133
study 0.0006306912375209815
row 0.0006303861382089941
followed 0.0006285116121270782
specially 0.0006283258726084885
exceeds 0.00062798259432602
rubber 0.0006277769231506801


topic 2 : [0. 0. 1. 0. 0.]
casing 0.0006366927137877419
c

The output shows the probability assigned to each word for ever topic present.

In [19]:
topics_reviews = defaultdict(list)
#topic_words
def get_topic(topic):
    return list([df["Text"][i] for i in range(len(df['Text'])) if any(ele in df["Text"][i] for ele in topic_words[topic])])
topics_reviews = {topic: get_topic(topic)  for topic in range(0,5)}

for i in topics_reviews:
    print("\n\ntopic #"+str(i))
    for text in topics_reviews[i]:
        print(text)
    



topic #0
Best I've found so far .... I've tried 2 other bluetooths and this one has the best quality (for both me and the listener) as well as ease of using.
I love my 350 headset.. My Jabra350 bluetooth headset is great, the reception is very good and the ear piece is a comfortable fit.
I know that sounds funny, but to me it seemed like sketchy technology that wouldn't work well.Well, this one works great.
This phone is very fast with sending any kind of messages and web browsing is significantly faster than previous phones i have used.
Nice quality build, unlike some cheap s*** out there.
The file browser offers all the options that one needs.Handsfree is great.
It is very comfortable to wear as well, which is probably the most important aspect about using a case.
Also, if your phone is dropped, this case is not going to save it, specially when dropped face down.
I especially love the long battery life.
Another note about this phone's appearance is that it really looks rather bland